# Setup AML Workspace
---

This notebook walks you through all the necessary steps to configure your environment for this solution accelerator including:

1. Connecting to your workspace 
2. Create a config.json (this can be skipped if running on an AML compute instance)
3. Create a compute cluster


## 1.0 Connect to workspace

Connect this solution accelerator to your AML workspace. This step isn't necessary if you're using a Notebook VM.

The following cell allows you to specify your workspace parameters. This cell uses the python method os.getenv to read values from environment variables which is useful for automation. If no environment variable exists, the parameters will be set to the specified default values.

In [10]:
import os
import datetime

#Set Environment Variables
os.environ['AZURE_TENANT_ID']="<Azure Tenant ID>"
os.environ['AZURE_CLIENT_ID']="<Azure Client ID>"
os.environ['AZURE_CLIENT_SECRET']="<Azure Client Secret>"

subscription_id=os.getenv("SUBSCRIPTION_ID", default="<Subscription ID>")
resource_group=os.getenv("RESOURCE_GROUP", default="<Resource Group>")
workspace_name=os.getenv("WORKSPACE_NAME", default="<Workspace Name>")
workspace_region=os.getenv("WORKSPACE_REGION", default="<Workspace Region>")

In [11]:
# Set Environment variables: https://docs.microsoft.com/en-us/azure/key-vault/general/tutorial-python-virtual-machine?tabs=azure-cli

from azure.ai.ml import MLClient
from azure.ai.ml.entities import Workspace
from azure.identity import DefaultAzureCredential
import datetime

ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group)

basic_workspace_name = "mlw-basic-prod-" + datetime.datetime.now().strftime(
    "%Y%m%d%H%M"
)

ws_basic = Workspace(
    name=basic_workspace_name,
    location="australiaeast",
    display_name="Basic workspace-example",
    description="This example shows how to create a basic workspace",
    hbi_workspace=False,
    tags=dict(purpose="demo"),
)

ml_client.workspaces.begin_create(ws_basic)

The deployment request mlw-basic-prod-202209110348-2819687 was accepted. ARM deployment URI for reference: 
https://portal.azure.com//#blade/HubsExtension/DeploymentDetailsBlade/overview/id/%2Fsubscriptions%2F80a3336a-33ac-4098-a7e7-64eb71d80cee%2FresourceGroups%2Ftgrgml%2Fproviders%2FMicrosoft.Resources%2Fdeployments%2Fmlw-basic-prod-202209110348-2819687
Creating Storage Account: (mlwbasicstoragecf2cc6d6e  ) ....  Done (30s)
Creating AppInsights: (mlwbasicinsights0553c757  )  Done (5s)
Creating KeyVault: (mlwbasickeyvaultbbcc48aa  )  Done (19s)
Creating workspace: (mlw-basic-prod-202209110348  ) ....  Done (32s)
Total time : 1m 4s


Workspace({'discovery_url': 'https://australiaeast.api.azureml.ms/discovery', 'mlflow_tracking_uri': 'azureml://australiaeast.api.azureml.ms/mlflow/v1.0/subscriptions/80a3336a-33ac-4098-a7e7-64eb71d80cee/resourceGroups/tgrgml/providers/Microsoft.MachineLearningServices/workspaces/mlw-basic-prod-202209110348', 'name': 'mlw-basic-prod-202209110348', 'description': 'This example shows how to create a basic workspace', 'tags': {'purpose': 'demo', 'createdByToolkit': 'sdk-v2-2.4.1'}, 'properties': {}, 'id': '/subscriptions/80a3336a-33ac-4098-a7e7-64eb71d80cee/resourceGroups/tgrgml/providers/Microsoft.MachineLearningServices/workspaces/mlw-basic-prod-202209110348', 'base_path': './', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f20fdc8eb30>, 'display_name': 'Basic workspace-example', 'location': 'australiaeast', 'resource_group': 'tgrgml', 'hbi_workspace': False, 'storage_account': '/subscriptions/80a3336a-33ac-4098-a7e7-64eb71d80cee/resourceGroups/tgr

In [12]:
# ML Client retreives a list of workspaces within the resource group:

for ws in ml_client.workspaces.list():
    print(ws.name, ":", ws.location, ":", ws.description)

mlw-basic-prod-202209110348 : australiaeast : This example shows how to create a basic workspace


## 2.0 Create compute cluster

You will need a compute cluster for training and batch forecasting.
This is a one-time set up so you won't need to re-run this in future notebooks, but you'll need to use the same cluster name to reference this cluster in following notebooks.

In [5]:
# Choose a name for your CPU cluster
cpu_cluster_name = 'cpucluster'

We create a STANDARD_D13_V2 compute cluster. D-series VMs are used for tasks that require higher compute power and temporary disk performance. This [page](https://docs.microsoft.com/azure/cloud-services/cloud-services-sizes-specs) will give you more information on VM sizes to help you decide which will best fit your use case.

In [27]:
from azure.ai.ml.entities import ComputeInstance, AmlCompute

workspace = ws_basic.name

ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

cpu_target = "tg_cpu_cluster"

try:
    ml_client.compute.get(cpu_target)
except Exception:
    compute = AmlCompute(
        name="tg-cpu-cluster",
        type="amlcompute",
        size="STANDARD_DS3_v2",
        location="australiaeast",
        min_instances=0,
        max_instances=2,
        idle_time_before_scale_down=120,
    )
    ml_client.begin_create_or_update(compute)

AmlCompute({'type': 'amlcompute', 'created_on': None, 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'tg-cpu-cluster', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/80a3336a-33ac-4098-a7e7-64eb71d80cee/resourceGroups/tgrgml/providers/Microsoft.MachineLearningServices/workspaces/mlw-basic-prod-202209110348/computes/tg-cpu-cluster', 'base_path': './', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f20fc9e6050>, 'resource_id': None, 'location': 'australiaeast', 'size': 'STANDARD_DS3_V2', 'min_instances': 0, 'max_instances': 2, 'idle_time_before_scale_down': 120.0, 'identity': None, 'ssh_public_access_enabled': True, 'ssh_settings': None, 'network_settings': <azure.ai.ml.entities._compute.compute.NetworkSettings object at 0x7f20fc9e61d0>, 'tier': 'dedicated'})

## Next Steps

Now that the AML Workspace its configured, it's time to create the sample datasets. Follow the steps in [01_Data_Preparation.ipynb](01_Data_Preparation.ipynb) for that.